# Basics of Agents
We shall look at the basics of the Chat Completions API and prompting which are the underpinnings of 
Agentic operations such as tool calls.

In [ ]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [110]:
from openai import OpenAI
client = OpenAI()

# https://platform.openai.com/docs/api-reference/chat/create

TEST_PROMPT = """
You are an helpful AI Assistant, with the ability to fetch restaurants and answer questions based on provided restaurant
reviews. Use the tool calling to fetch reviews for a given restaurant requested by the user.
When the reviews are given as a list, copy them to the output for the next AI assistant to work on. Do not summarize.

Tool Call: fetch_review(restaurant_name) where restaurant_name is the name of the restaurant. Do not output anything else.
"""


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": TEST_PROMPT},
        {
            "role": "user",
            "name": "Joe",
            "content": "Fetch reviews for Starbucks."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='fetch_review(restaurant_name="Starbucks")', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [111]:
print(completion.choices[0].message.content)

fetch_review(restaurant_name="Starbucks")


In [94]:
import agentops

from autogen import ConversableAgent

In [95]:
agentops.init(os.environ.get("AGENTOPS_API_KEY"))

# Microsoft Autogen: ConversableAgent: https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction/
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


In [96]:
chat_result = joe.initiate_chat(cathy, message="Tell me a joke", max_turns=2)

joe (to cathy):

Tell me a joke

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's one: 

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears, and the beans stalk!

--------------------------------------------------------------------------------
joe (to cathy):

Good one, Cathy! Here's another one for you:

Why don’t scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Oh, Joe, that's a good one! A little science humor. Here's another chemistry joke for you:

Why can't you trust an atom that's been split?

Because it's gone fission!

--------------------------------------------------------------------------------


In [53]:
from typing import Annotated, Literal
from typing import Dict
from typing import List
restaurant_datafile="restaurant-data.txt"


def load_restaurant_reviews() -> Dict[str, List[str]]:
    restaurant_map :Dict[str, List[str]] = {}
    with open(restaurant_datafile) as file:
        for line in file:
            # print(line)
            splits = line.split(".")
             # Use setdefault to ensure the key has a list, then append the value
            restaurant_map.setdefault(splits[0], []).append(splits[1])
    # Iterate through the dictionary
    # for key, value in restaurant_map.items():
    #    print(f"Key: {key}, Value: {value}")
    return restaurant_map
    
def fetch_restaurant_reviews(restaurant_name: Annotated[str, "restaurant name"]) -> Dict[str, List[str]]:
    reviews = load_restaurant_reviews()
    return {restaurant_name: reviews[restaurant_name]}

In [54]:
fetch_restaurant_reviews("Starbucks")

{'Starbucks': [' Starbucks consistently serves good coffee and tasty pastries',
  ' The coffee was good and consistently prepared',
  ' Starbucks consistently delivers good coffee and pastries',
  ' Starbucks consistently serves good coffee and tasty pastries',
  ' Both the food and service at Starbucks were great',
  ' Starbucks consistently serves good coffee and tasty snacks',
  ' Starbucks consistently delivers good coffee and pastries',
  ' Starbucks offers good coffee and a variety of enjoyable snacks',
  ' Starbucks consistently delivers good coffee and snacks',
  ' Starbucks provided good quality coffee and pastries',
  ' Starbucks consistently serves enjoyable coffee and pastries',
  ' Starbucks consistently serves good coffee and pastries',
  ' Starbucks consistently serves good coffee and snacks',
  ' Starbucks provided good quality coffee and tasty snacks',
  ' Starbucks provided a satisfying coffee experience with good pastries',
  ' Starbucks consistently delivers good co

In [112]:
ENTRYPOINT_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to fetch restaurants and answer questions based on provided restaurant
reviews. Use the tool calling to fetch reviews for a given restaurant requested by the user.
When the reviews are given as a list, copy them to the output for the next AI assistant to work on. Do not summarize.
"""

restaurant_review_agent = ConversableAgent("Restaurant Review agent",
    system_message=ENTRYPOINT_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)


# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)



In [113]:
# Tool use: https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use/
# Register the tool signature with the assistant agent.
restaurant_review_agent.register_for_llm(name="fetch_restaurant_reviews",
                                         description="Fetch reviews for a given restaurant name.")(fetch_restaurant_reviews)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="fetch_restaurant_reviews")(fetch_restaurant_reviews)

<function __main__.fetch_restaurant_reviews(restaurant_name: typing.Annotated[str, 'restaurant name']) -> Dict[str, List[str]]>

In [114]:
# Termination: use max_turns = 2
chat_result = user_proxy.initiate_chat(restaurant_review_agent, message="Fetch reviews for Starbucks.", max_turns=2)

User (to Restaurant Review agent):

Fetch reviews for Starbucks.

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_Ki1uZvKMKpy2cjZAXaXQimih): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_Ki1uZvKMKpy2cjZAXaXQimih) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries", " The coffee was good and consistently prepared", " Starbucks consistently delivers good coffee and pastries", " Starbucks consistently serves good coffee and tasty pastries", " Both the food and service at Starbucks were grea

In [115]:
# Sequence patterns: https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns
FOODSCORE_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to analyze and rate (from 1 to 5) the food service at a restaurant given the reviews.
Use the following table:
Score 1/5 has one of these adjectives: awful, horrible, or disgusting.
Score 2/5 has one of these adjectives: bad, unpleasant, or offensive.
Score 3/5 has one of these adjectives: average, uninspiring, or forgettable.
Score 4/5 has one of these adjectives: good, enjoyable, or satisfying.
Score 5/5 has one of these adjectives: awesome, incredible, or amazing.
Given a review, use the above table to output a rating as an integer. Only output the integer and nothing else.
"""

food_score_agent = ConversableAgent("Food Score agent",
    system_message=FOODSCORE_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [73]:
chat_result = user_proxy.initiate_chats(
[
        {
            "recipient": restaurant_review_agent,
            "message": "Whats the overall score for Starbucks ?",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": food_score_agent,
            "message": "Given these reviews, generate a score for the food.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
])


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Restaurant Review agent):

Whats the overall score for Starbucks ?

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_mydGs6sdB1erl0if7eHiPCFW): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_mydGs6sdB1erl0if7eHiPCFW) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries", " The coffee was good and c

In [90]:
# Sequence patterns: https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns
REVIEWSCORE_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to analyze and rate (from 1 to 5) the food service and customer service at a restaurant given the reviews.
Use the following table:
Score 1/5 has one of these adjectives: awful, horrible, or disgusting.
Score 2/5 has one of these adjectives: bad, unpleasant, or offensive.
Score 3/5 has one of these adjectives: average, uninspiring, or forgettable.
Score 4/5 has one of these adjectives: good, enjoyable, or satisfying.
Score 5/5 has one of these adjectives: awesome, incredible, or amazing.
Given a list of individual reviews, use the above table to output a rating as an integer along with the review. Output one integer each separately for food service and one for
customer service. Provide your output as a list of tuples (x,y, z) where x is the food service rating and y is the customer service rating for the review z.
"""

review_score_agent = ConversableAgent("Review Score agent",
    system_message=REVIEWSCORE_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)


In [91]:
chat_result = user_proxy.initiate_chats(
[
        {
            "recipient": restaurant_review_agent,
            "message": "Whats the overall score for Starbucks ?",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": review_score_agent,
            "message": "Given these reviews, generate separate scores for the food and customer service.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
])


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Restaurant Review agent):

Whats the overall score for Starbucks ?

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_MS1CYsivOetITdyDB5L5Ce48): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_MS1CYsivOetITdyDB5L5Ce48) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries", " The coffee was good and c